In [ ]:
#  Finds the MDP transition matrix on the train set

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
data = pd.read_csv("../data/rl_train_data_final_discrete.csv")

In [6]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [7]:
#make the transition distribution a dictionary of {(state, action) : {next_state : count }}
trans_counts = {}
count = 0
for row in data.index:
    cur_state = data.ix[row, 'state']
    
    # extract the iv and vaso dose administered, and from that the action id
    iv = int(data.ix[row, 'iv_tev_in'])
    vaso = int(data.ix[row, 'max_dose_vaso'])
    action = action_map[iv,vaso]
    
    # get the next state, if it exists
    # first case handles the very end of the dataframe, edge case
    if row == len(data)-1:
        if data.ix[row, 'mortality'] == 0:
            next_state = 'END_REWARD'
        else:
            next_state = 'END_PENALTY'
    # second case is for all other rows
    else:
        if data.ix[row, 'icustayid'] == data.ix[row+1, 'icustayid']:
            next_state = data.ix[row+1, 'state']
        else:
            # reached the end for this patient
            if data.ix[row, 'mortality'] == 0:
                next_state = 'END_REWARD'
            else:
                next_state = 'END_PENALTY'
    
    # add this transition to trans_counts
    if (cur_state, action) in trans_counts:
        if next_state in trans_counts[(cur_state,action)]:
            trans_counts[(cur_state,action)][next_state] += 1
        else:
            trans_counts[(cur_state,action)][next_state] = 1
    else:
        # (state,action) not in trans_counts
        trans_counts[(cur_state,action)] = {next_state : 1}
    count += 1
#     print "current row \n", data.ix[row]
#     print "iv, vaso" , iv, vaso
#     print "next row \n", data.ix[row+1]
#     print "cur state, action, next state", cur_state,action, next_state
#     print trans_counts[(cur_state,action)]
#     print "\n"
#     if count > 15:
#         break

In [8]:
# normalise the transition counts
trans_prob = {}
count = 0
for key, emp_dist in trans_counts.items():
    count += 1
    norm_const = sum(emp_dist.values())
    trans_prob[key] = {}
#     print key
    for next_state, count in emp_dist.items():
        trans_prob[key][next_state] = float(count)/float(norm_const)
#     print trans_counts[key]
#     print trans_prob[key]
#     if count > 20:
#         break

In [9]:
# pickle the trans counts to file for the value iteration stage later
import cPickle as pickle
with open('trans_prob.p', 'wb') as f:
    pickle.dump(trans_prob, f)

In [10]:
count = 0
for i in trans_prob:
    print i, trans_prob[i]
    count += 1
    if count > 10:
        break

(360, 14) {360: 1.0}
(660, 3) {735: 1.0}
(1153, 21) {195: 1.0}
(536, 17) {536: 0.6666666666666666, 1139: 0.3333333333333333}
(114, 5) {996: 0.05555555555555555, 'END_PENALTY': 0.05555555555555555, 652: 0.05555555555555555, 850: 0.05555555555555555, 1134: 0.05555555555555555, 114: 0.5555555555555556, 853: 0.05555555555555555, 247: 0.05555555555555555, 857: 0.05555555555555555}
(167, 13) {167: 1.0}
(228, 13) {228: 1.0}
(157, 23) {157: 0.5, 959: 0.5}
(450, 3) {450: 1.0}
(512, 10) {512: 0.5, 469: 0.25, 725: 0.25}
(440, 21) {16: 0.25, 81: 0.25, 440: 0.25, 374: 0.25}
